# Cisco IOS CLI

En este cuaderno se presentan los principales comandos del IOS CLI (Command Line Interface) de los equipos de Cisco.

Realizado por:

Ferney O. Amaya Fernández <BR>
ferney.amaya@gmail.com


##1. Conexión a consola

Para conectarnos desde una terminal de Linux a la consola del equipo empleamos la aplicación putty con el siguiente comando:

> ```putty /dev/ttyUSB0 -serial -sercfg 9600,8,n,1,N```

##2. Modo usuario, privilegiado y global

Para pasar del modo usuario (>) al modo privilegiado (#):
> ```enable```

Para pasar del modo privilegiado (#) al modo global (config):

> ```	#configure terminal```


##3. Observar y eliminar la configuración

Los comandos ingresados quedan en la RAM del dispositivo: running-config. Para observar los comandos:
> ```#show running-config```

Para guardar la configuración en la NVRAM:
>```#copy running-config startup-config```

Para observar la configuración almacenada en la NVRAM:
>```#show startup-config```

Para borrar la configuración del switch, utilice el comando:
>```#erase startup-config```

Para reiniciar desde la NVRAM:
>```#reload```

Cuando pregunte si desea entrar al diálogo de configuración, escriba N y presione Intro.

##4. Asignar nombre de host (hostname) y clave del modo privilegiado

Asignar nombre a host: Por ejemplo, asignar el nombre Redes:
>```(config)#hostname Redes```

Asignar clave de modo privilegiado: Asignar la clave “class”:
>```(config)#enable secret class```

##5. Asignar una IP a una interface

Por ejemplo, asignar la IP 10.10.10.1/8 a la interface FastEthernet 0/0:
>```(config)#interface FastEthernet 0/0```<BR>
>```(config-if)#ip address 10.10.10.1 255.0.0.0```<BR>
>```(config-if)#no shutdown```<BR>
>```(config-vlan)# exit```<BR>

##6. Configurar el protocolo RIP versión 2

Por ejemplo, se quiere advertir de las redes 10.0.0.0/8 y 20.20.0.0/16 a las que está conectado en enrutador:

>```(config)#router rip```<BR>
>```(config-router)#version 2```<BR>
>```(config-router)#network 10.0.0.0```<BR>
>```(config-router)#network 20.20.0.0```<BR>
>```(config-vlan)# exit```<BR>

Para observar la tabla de enrutamiento:
>```#show ip route```<BR>

##7. Configurar VLANs

###Crear una VLAN en el switch
El rango normal es de 1 a 1005, estando reservadas las VLAN: 1, 1002 - 1005.Por ejemplo, para crear la VLAN 10 y asignarle al nombre Admin:

>```(config)# vlan 10```<BR>
>```(config-vlan)# name Admin```<BR>
>```(config-vlan)# exit```<BR>

###Configurar los puertos de acceso en el switch
Por ejemplo, asignar los puertos 2 a 15 a la VLAN 10:

>```(config)#interface range fa0/2-15```<BR>
>```(config-if-range)#switchport mode access```<BR>
>```(config-if-range)#switchport access vlan 10```<BR>
>```(config-if-range)# exit```

###Configurar un puerto troncal en el switch
Por ejemplo, configurar el puerto 24 como troncal, VLAN nativa la 400 y permitir las VLANs 10,20,30:

>```(config)#interface fa 0/24```<BR>
>```(config-if)#switchport mode trunk```<BR>
>```(config-if)#switchport trunk native vlan 400```<BR>
>```(config-if)#switchport trunk allowed vlan 10,20,30```<BR>

###Configurar VLANs en una interface del Router (Router on a stick)
Por ejemplo, si a la interface FastEthernet 0/0 le asignamos la VLAN 10 con la IP 200.10.0.1/16:

>```(config)#interface fa 0/0.10```<BR>
>```(config-subif)#encapsulation dot1q 10```<BR>
>```(config-subif)#ip address 200.10.0.1 255.255.0.0```<BR>
>```(config-subif)#exit```<BR>

###Verificar VLANs en el Switch
Para verificar las VLANs y asignaciones de puerto:
>```#show vlan```<BR>

Para verificar las troncales:
>```#show interface trunk```<BR>

##8. Servidor DHCP

###Establecer el rango de IPs excluido
El rango de IPs excluido son las IPs que el servidor no entregará. Por ejemplo, si el rango excluido es desde la 200.10.10.1 hasta la 200.10.10.5:

>```(config)#ip dhcp excluded 200.10.10.1 200.10.10.5```<BR>

###Configurar el pool de direcciones IP
Por ejemplo, si la red para entregar IPs es la 200.10.10.0/ 24 y la puerta de enlace e 200.10.10.1 y al pool lo llamamos Admon:
>```(config)#ip dhcp pool Admon```<BR>
>```(dhcp-config)#network 200.10.10.0 255.255.255.0```<BR>
>```(dhcp-config)#default-router 200.10.10.1```<BR>

Si adicionalmente se quiere configurar el servidor DNS, por ejemplo, si tiene la IP 200.10.10.5:
>```(dhcp-config)#dns-server 200.10.10.5```<BR>

##9. IP-Helper, DHCP
Esta opción se usa cuando el servidor DHCP se encuentra en una red diferente a donde se encuentran los dispositivos que solicitarán direcciones IP. En este caso, el enrutador recibe las solicitudes por la interface de los dispositivos solicitantes y las envía al servidor DHCP. Por ejemplo, si los solicitantes están conectados a la interface fa 0/0 y la IP del servidor DHCP, que se encuentra en otra red es la 30.30.30.5, la configuración se realiza dentro de la interface del enrutador:
>```(config)#interface fa 0/0```<BR>
>```(config-if)#ip helper-address 30.30.30.5```<BR>

##10. Servidor NAT

###NAT Estático
Por ejemplo, si la IP local es 10.10.10.100 y la IP global es 200.200.200.100:
>```(config)# ip nat inside source static local-ip global-ip```<BR>

###NAT Dinámico
En este caso hay que creear un pool NAT. Por ejemplo, se tienen los siguientes parámetros:
- Nombre del pool NAT: NAT
- IP inicial del pool NAT: 200.2.2.1
- IP final del pool NAT: 200.2.2.8
- Red del pool NAT: 200.2.2.0/24
- Máscara 255.255.255.0
- Red local: 10.0.0.0/8
- Máscara red local: 255.0.0.0

Debe calcularse el wildcard para el pool NAT, que es el inverso de la máscara, en este caso es 0.0.0.255.

Los comandos para realizar la configuración son:

>```ip nat pool NAT 200.2.2.1 200.2.2.8 netmask 255.255.255.0```<BR>
>```access-list 1 permit 10.0.0.0 255.0.0.0```<BR>
>```ip nat inside source list 1 pool NAT```<BR>

###PAT
En este caso también hay que creear un pool NAT. Por ejemplo, se tienen los siguientes parámetros:
- Nombre del pool NAT: PAT
- IP inicial del pool NAT: 200.2.2.1
- IP final del pool NAT: 200.2.2.8
- Red del pool NAT: 200.2.2.0/24
- Máscara 255.255.255.0
- Red local: 10.0.0.0/8
- Máscara red local: 255.0.0.0

Debe calcularse el wildcard para el pool NAT, que es el inverso de la máscara, en este caso es 0.0.0.255.

Los comandos para realizar la configuración son:

>```ip nat pool PAT 200.2.2.1 200.2.2.8 netmask 255.255.255.0```<BR>
>```access-list 1 permit 10.0.0.0 255.0.0.0```<BR>
>```ip nat inside source list 1 pool PAT overload```<BR>

###Definición de interfaces

Este paso debe realizarse para todas las variantes de NAT: estático, dinámico o por sobrecarga y consiste en indicarle al enrutador, cuáles son las interfaces donde están conectadas las redes interna y externa.

Por ejemplo, si la red interna está conectada a la interface fastEthernet 0/0 y la externa a la serial 0/0, la configuración es

>```(config)#interface fastEthernet 0/0```<BR>
>```(config-if)#ip nat inside```<BR>
>```(config-if)#exit```<BR>
>```(config)#interface serial 0/0```<BR>
>```(config-if)#ip nat outside```<BR>

###Verificar NAT

Puede verificarse la configuración con los siguientes comandos:

>```#show ip nat translations```<BR>
>```#show ip nat statistics```<BR>

##11. ACLs

###ACLs estándar
Permiten bloquear el tráfico proveniente de una fuente. Emplea los números del 0 al 99 pueden usarse:

Definir la ACL:
>```(config)#access-list number {permit|deny} {source [wildcard]}```<BR>

Aplicar la ACL a la interface:
>```(config-if)#ip access-goup acl-number {in|out}```<BR>

Ejemplo:
>```(config)#access-list 2 deny 172.16.1.1```<BR>
>```(config)#access-list 2 permit 172.16.1.0 0.0.0.255```<BR>
>```(config)#interface fa 0/0```<BR>
>```(config-if)#ip access-group 2 in```<BR>

###ACLs extendida
Permiten especificar más parámetros. Emplea los números del 100 al 199. Puede emplearse:
- Operaciones lógicas: equal (eq), not equal (neq), greater than (gt), and less than (lt).
- Protocolos: ip, tcp, udp, icmp.

Definir la ACL:
>```(config)#access-list number {permit|deny} protocol source wildcard destination wildcard {condicion}```<BR>

Aplicar la ACL a la interface:
>```(config-if)#ip access-group acl-number {in|out}```<BR>

Ejemplo:
>```(config)#access-list 114 permit tcp 172.16.6.0 0.0.0.255 any eq telnet```<BR>
>```(config-if)#ip access-group 114 in```<BR>

###Verificar ACLs
Emplear los comandos:

>```show access-lists```<BR>
>```show ip interface```<BR>
